
# Datasets currently not used



## [The COVID tracking project testing data.](#source1)
[https://covidtracking.com/api](https://covidtracking.com/api)

## [Delphi-epidata ](#delphi) which contains 
       Facebook surveys, google surveys, doctor visits, google health trends, quidel test data
[https://cmu-delphi.github.io/delphi-epidata/api/covidcast.html](https://cmu-delphi.github.io/delphi-epidata/api/covidcast.html)

I have not dove into this dataset too thoroughly but it contains information from facebook and google
surveys regarding COVID as well as doctor visits; the doctor visit data attempts to make distinctions between
those sick with the annual influenza and those with COVID.


## [IHME hospital data](#ihme)

**Data available at:**
[http://www.healthdata.org/covid/data-downloads](http://www.healthdata.org/covid/data-downloads)

The IHME hospital data is one of the more unique datasets I've discovered with 



In [ ]:
def normalize_Xy_splits(splits, normalization_method='minmax', train_test_only=False, feature_indices=None):
    """ Split into training, validation and test data.
    """
    min_, max_ = (0, 0.5)
    # Note that the last frame (date_range) that exists in X has already been determined by the choice of the number
    # of steps to predict in the future, this is only slicing the frames. 
    if train_test_only:
        (X_train, y_train, X_test, y_test) = splits
    else:
        (X_train, y_train, X_validate, y_validate, X_test, y_test) = splits

#     if normalization_method=='minmax':
        # To encapsulate the time-dependent nature of the problem and ignore the dramatic difference between current
        # and initial behavior, only rescale the validation and testing frames by the most recent frame's values.
        # There is only a single value per feature in this case, meaning that to rescale, the values need to
        # be repeated for each validation, test frame for each country for each timestep.
    X_min = X_train.min(axis=(2))
    X_max = X_train.max(axis=(2))


    X_train_scaled = minmax(X_train, X_min[:, :, np.newaxis, :],
                    X_max[:, :, np.newaxis, :])
    X_test_scaled = minmax(X_test, X_min[-1][np.newaxis, :, np.newaxis, :], 
                           X_max[-1][np.newaxis, :, np.newaxis, :])
    if train_test_only:
    # Normalize the training data by each frame's specific mean and std deviation. 
        splits = (X_train_scaled, y_train, X_test_scaled, y_test)
    else:
        X_validate_scaled = minmax(X_validate, X_min[-1,:][np.newaxis, :, np.newaxis, :], 
                                   X_max[-1,:][np.newaxis, :, np.newaxis, :])
        splits = (X_train_scaled, y_train, X_validate_scaled, y_validate, X_test_scaled, y_test)
#     else:
        
#         X_mean = X_train.mean(axis=(1,2))
#         X_std = X_train.std(axis=(1,2))


#         X_train_scaled = normal(X_train, 
#                                 X_mean[:, np.newaxis, np.newaxis, :],
#                                 X_std[:, np.newaxis, np.newaxis, :])
#         X_test_scaled = normal(X_test, 
#                                X_mean[-1,:][np.newaxis, np.newaxis, np.newaxis, :], 
#                                X_std[-1,:][np.newaxis, np.newaxis, np.newaxis, :])
        
#         if train_test_only:
#         # Normalize the training data by each frame's specific mean and std deviation. 
#             splits = (X_train_scaled, y_train, X_test_scaled, y_test)
#         else:
#             X_validate_scaled = normal(X_test, 
#                                X_mean[-1,:][np.newaxis, np.newaxis, np.newaxis, :], 
#                                X_std[-1,:][np.newaxis, np.newaxis, np.newaxis, :])
#             splits = (X_train_scaled, y_train, X_validate_scaled, y_validate, X_test_scaled, y_test)

    return splits

## IHME hospital data
<a id='ihme'></a>
[Return to table of contents](#toc)

[JHU CSSE](#csse) 
<font color='red'>
### Has all USA states but only 32 countries which overlap with other data; stash this dataset for now. 
</font>


In [ ]:
ihme_df = column_or_index_string_reformat(pd.read_csv(
    './IHME_hospital_data/2020_04_12.02/Hospitalization_all_locs.csv').rename(columns={'location_name':'location'}))
ihme_df.loc[:, 'date'] = pd.to_datetime(ihme_df.loc[:,'date']).dt.normalize()
ihme_df = ihme_df.set_index(['location', 'date']).sort_index()
ihme_df.sample(5)

## Delphi-epidata
<a id='delphi'></a>
[Return to table of contents](#toc)

data_source	name of upstream data source 
(e.g., fb-survey, google-survey, ght, quidel, doctor-visits)	string

signal	name of signal derived from upstream data (see notes below)	string

time_type	temporal resolution of the signal (e.g., day, week)	string

geo_type	spatial resolution of the signal (e.g., county, hrr, msa, dma, state)	string

time_values	time unit (e.g., date) over which underlying events happened	list of time values (e.g., 20200401)

geo_value	unique code for each location, depending on geo_type (county -> FIPS 6-4 code, HRR -> HRR number, MSA -> CBSA code,
DMA -> DMA code, state -> two-letter state code), or * for all	string

As of this writing, data sources have the following signals:

fb-survey signal values include raw_cli, raw_ili, raw_wcli, raw_wili, and also four additional named with raw_* replaced by smoothed_* (e.g. smoothed_cli, etc).
google-survey signal values include raw_cli and smoothed_cli.
ght signal values include raw_search and smoothed_search.
quidel signal values include smoothed_pct_negative and smoothed_tests_per_device.
doctor-visits signal values include smoothed_cli.

Delphi API data :
doctor visits : 20200201-20200429 (as of 20200503)


In [ ]:
#----------------- Currently Unused ----------------------#

def pull_delphi_data(data_source=['fb-survey', 'google-survey', 'ght', 'quidel', 'quidelneg', 'doctor-visits'], 
                     daterange=pd.date_range(start="20200101",
                                             end=''.join(str(datetime.now().date()).split('-'))).strftime('%Y%m%d'),
                     **kwargs):
    """ Pull data from https://cmu-delphi.github.io/delphi-epidata/api/
        https://cmu-delphi.github.io/delphi-epidata/api/covidcast.html
    
    
    
    """
    
    for data in data_source:
        signal_dict = {'fb-survey':'smoothed_cli',
                       'google-survey':'smoothed_cli',
                       'ght':'smoothed_search',
                       'quidel':'smoothed_tests_per_device',
                       'quidelneg':'smoothed_pct_negative',
                       'doctor-visits':'smoothed_cli'}
        
        signal = signal_dict[data]
        if data=='quidelneg':
            #change the proxy for the quidel signal
            data = 'quidel'
        for days in daterange:
            resp = requests.get('https://delphi.cmu.edu/epidata/api.php?source=covidcast&data_source=doctor-visits&signal=smoothed_cli&time_type=day&geo_type=county&geo_value=*&time_values='+days)
            day_data = resp.json().get('epidata', None)
            if day_data is None:
                pass
            else:
                var_number += pd.json_normalize(day_data).size
                print(pd.json_normalize(day_data).shape)    
                
                
# date_range_2020 = pd.date_range(start="20200101",end=''.join(str(datetime.now().date()).split('-'))).strftime('%Y%m%d')
# var_number = 0 
# for days in date_range_2020:
# #     days='20200302'
#     resp = requests.get('https://delphi.cmu.edu/epidata/api.php?source=covidcast&data_source=doctor-visits&signal=smoothed_cli&time_type=day&geo_type=county&geo_value=*&time_values='+days)
#     day_data = resp.json().get('epidata', None)
#     if day_data is None:
#         pass
#     else:
#         var_number += pd.json_normalize(day_data).size
#         print(pd.json_normalize(day_data).shape)

# <font color='red'>Unused as of now</font>


## Repeat of the above calculations for United States only data.



The United States' data merits separate investigation 1. because of the case number 2. because the IHME dataset is only really
properly defined for the statewide description of the U.S. 

In [ ]:
csse_us_timeseries_df.groupby(level=1).sum().to_csv('csse_united_states.csv')

usa_data = [
    csse_us_timeseries_df,
    ihme_df,
    owid_df,
    oxcgrt_df,
    testtrack_df]

us_table = multiindex_to_table(csse_us_timeseries_df)
today = us_table[us_table.date==us_table.date.max()]

In [ ]:
# unused
#Pull the data using their API (for whatever reason this data set is different from the manual download).
# url_to_present_date = 'https://covidtrackerapi.bsg.ox.ac.uk/api/v2/stringency/date-range/2020-01-02/' \
#                         + str(datetime.now().date())
# response = requests.get(url_to_present_date)
# response_json = response.json()
# response_json_nested_dict = response_json['data']

# response_api_df = pd.DataFrame.from_dict({(i,j): response_json_nested_dict[i][j] 
#                            for i in response_json_nested_dict.keys() 
#                            for j in response_json_nested_dict[i].keys()},
#                        orient='index')

In [ ]:
x = np.arange(-10, 10, 0.25)
sigmoid = 1.0 / (1.0 + np.exp(-x))

A = np.vstack([x, np.ones(len(x))]).T
m, c = np.linalg.lstsq(A, sigmoid, rcond=None)[0]

over = np.where(((m * x + c - sigmoid) > 0)  & (x<=0))
under = np.where(((m * x + c - sigmoid) < 0) & (x>=0))
xover_width = x[over].max() - x[over].min() 
xunder_width = x[under].max() - x[under].min()

sigmoidmin, sigmoidmax = [-0.05, 1.05]

# fig, (ax,ax2) = plt.subplots(1, 2, figsize=(10, 5))
fig, ax = plt.subplots(figsize=(10, 5))

_ = ax.plot(x, sigmoid, color='k')
_ = ax.plot(x, m * x + c, '--',color='k')
_ = ax.set_ylim([-0.05, 1.05])
_ = ax.vlines(x[over].min(), ymin=sigmoidmin, ymax=sigmoidmax)
_ = ax.vlines(0, ymin=sigmoidmin, ymax=sigmoidmax)
_ = ax.vlines(x[under].max(), ymin=sigmoidmin, ymax=sigmoidmax)
_ = ax.hlines(0, xmin=-10, xmax=10)
_ = ax.add_patch(Rectangle((x[over].min(), 0), xover_width, sigmoidmax, 
                           angle=0.0, alpha=0.5, color='k', label='Overestimation'))
_ = ax.add_patch(Rectangle((0, 0), x[under].max(), sigmoidmax, 
                           angle=0.0, alpha=0.25, color='gray', label='Underestimation'))

_ = ax.set_title('Possible explanation for regression prediction behavior')
_ = ax.legend(loc=(0,1.05))

In [ ]:
df_list = []

#the actual format difference is being covered up by pd.concat which fills with Nans
for x in glob.glob('COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/*.csv'):
    tmp_df = pd.read_csv(x)
    tmp_df.columns = reformat_values(tmp_df.columns, category='columns').values
    df_list.append(tmp_df)

daily_reports_df = pd.concat(df_list, axis=0)
daily_reports_df.columns = reformat_values(daily_reports_df.columns, category='columns').values
daily_reports_df.loc[:, 'date'] = reformat_values(daily_reports_df.loc[:, 'last_update'], category='date').values
daily_reports_df.loc[:, 'location'] =  reformat_values(daily_reports_df.loc[:, 'country_region'], category='location').values

daily_reports_df.loc[:, 'combined_key'] = (daily_reports_df.province_state.astype('str').replace(to_replace='nan', value='')+' '+ daily_reports_df.location.astype('str')).values

daily_reports_df = daily_reports_df.drop(columns=['province_state', 'last_update', 'fips', 'admin2']).set_index(['location','date'])

daily_reports_df.sample(5)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import sys
from matplotlib.patches import Rectangle
import re
sns.set()

# Introduction <a id='intro'></a>

This notebook uses a variety of different COVID-19 related datasets to explore the behavior
of the multiple time series'. This notebook also creates new features that attempt to encapsulate the
time dependent (and time delayed) nature of the problem; these will be used during the model creation
project which makes time dependent forecasting models. 


# Table of contents

## [Function definitions](#generalfunctions)

## [Data](#imports)

## [Exploratory Data Analysis](#EDA)

## [Feature production](#newfeatures)

## Function definitions <a id='generalfunctions'></a>

def rolling_means(df, features, roll_widths):
    new_feature_df_list = []
    for window in roll_widths:
        # order the dataframe so date is index, backfill in the first roll_width values 
        rollmean = pd.DataFrame(df.groupby(by='location').rolling(window).mean().fillna(value=0.))
#         rollstd = pd.DataFrame(df.groupby(by='location').rolling(window).std().fillna(value=0.))    
#         new_features = pd.concat((rollmean, rollstd), axis=1)
        new_features = rollmean
        new_cols = features +'_rolling_mean_' + str(window)
#         rsind = features +'_rolling_std_' + str(window)
#         new_cols = rmind.append(rsind)
        new_features.columns = new_cols
        new_feature_df_list.append(new_features)
    return new_feature_df_list

def tsplot(data, roll_width, **kw):
    rollmean = datatmp.rolling(roll_width).mean().fillna(method='backfill').values.ravel()
    rollstd  = datatmp.rolling(roll_width).std().fillna(method='backfill').values.ravel()
    cis = (rollmean - rollstd, rollmean + rollstd)
    fig, ax = plt.subplots()
    ax.fill_between(range(len(datatmp)), cis[0], cis[1], alpha=0.5)
    ax.plot(range(len(datatmp)), rollmean, color='k', **kw)
    return ax

def reformat_values(values_to_transform, category='columns',dateformat=None):
    """ Reformat column and index names. 
    
    Parameters :
    ----------
    df : Pandas DataFrame
    columns : bool
    index : bool
    
    Notes :
    -----
    Change headers of columns; this needs to be updated to account for their formatting changes. 
    This function converts strings with CamelCase, underscore and space separators to lowercase words uniformly
    separated with underscores. I.e. (hopefully!) following the correct python identifier syntax so that each column
    can be reference as an attribute if desired. 

    For more on valid Python identifiers, see:
    https://docs.python.org/3/reference/lexical_analysis.html#identifiers
    """
    
    """ Reformat column and index names. only works with with pandas MultiIndex for level=0.
    
    Parameters :
    ----------
    df : Pandas DataFrame

    Notes :
    -----
    Different datasets have different naming conventions (for countries that go by multiple names and abbreviations).
    This function imposes a convention on a selection of these country names.  
    """
    # these lists are one-to-one. countries compared via manual inspection, unfortunately. 
    mismatch_labels_bad = ['Lao People\'s Democratic Republic', 'Mainland China',
                           'Occupied Palestinian Territory','Republic of Korea', 'Korea, South', 
                           'Gambia, The ', 'UK', 
                           'USA', 'Iran (Islamic Republic of)',
                           'Bahamas, The', 'Russian Federation', 'Czech Republic', 'Republic Of Ireland',
                          'Hong Kong Sar', 'Macao Sar', 'Uk','Us',
                           'Congo ( Kinshasa)','Congo ( Brazzaville)',
                           'Cote D\' Ivoire', 'Viet Nam','Guinea- Bissau','Guinea','Usa']

    mismatch_labels_good = ['Laos','China',
                            'Palestine', 'South Korea', 'South Korea', 
                            'The Gambia', 'United Kingdom', 
                            'United States','Iran',
                            'The Bahamas','Russia','Czechia','Ireland',
                            'Hong Kong','Macao','United Kingdom', 'United States',
                            'Democratic Republic Of The Congo','Republic Of The Congo',
                            'Ivory Coast','Vietnam', 'Guinea Bissau','Guinea Bissau','United States']
    
    # three cases, column names, country names, or datetime. 
    if category == 'location':
        reformatted_values = []
        for val in values_to_transform:
            reformatted_values.append(' '.join(re.sub('([A-Z][a-z]+)', r' \1', 
                                                        re.sub('([A-Z]+)|_|\/|\)|\(', r' \1', val).lower())
                                                        .split()).title())
        transformed_values = pd.Series(reformatted_values).replace(to_replace=mismatch_labels_bad, value=mismatch_labels_good)
    
    elif category == 'columns':
        reformatted_values = []
        for val in values_to_transform:
            reformatted_values.append('_'.join(re.sub('([A-Z][a-z]+)', r' \1', 
                                                     re.sub('([A-Z]+)|_|\/|\)|\(', r' \1', val)
                                                            .lower()).split()))
        transformed_values = pd.Series(reformatted_values)
        
    elif category == 'date':
        transformed_values = pd.to_datetime(pd.Series(
            values_to_transform), errors='coerce',format=dateformat).dt.normalize()


    return transformed_values


def column_search(df, name):
    return df.columns[df.columns.str.contains(name)]

def country_groupby_indices_list(df):
    return [df[df.location==country].index for country in df.location.unique()]


def regularize_names(df, datekey=None, locationkey=None, dateformat=None):
    df.columns = reformat_values(df.columns, category='columns').values
    if datekey is not None:
        df.loc[:, 'date'] = reformat_values(df.loc[:, datekey], category='date', dateformat=None).values
    if locationkey is not None:
        df.loc[:, 'location'] =  reformat_values(df.loc[:, locationkey], category='location').values
    return df

def reformat_values(values_to_transform, category='columns',dateformat=None):
    """ Reformat column and index names. 
    
    Parameters :
    ----------
    df : Pandas DataFrame
    columns : bool
    index : bool|
    
    Notes :
    -----
    Change headers of columns; this needs to be updated to account for their formatting changes. 
    This function converts strings with CamelCase, underscore and space separators to lowercase words uniformly
    separated with underscores. I.e. (hopefully!) following the correct python identifier syntax so that each column
    can be reference as an attribute if desired. 

    For more on valid Python identifiers, see:
    https://docs.python.org/3/reference/lexical_analysis.html#identifiers
    """
    
    """ Reformat column and index names. only works with with pandas MultiIndex for level=0.
    
    Parameters :
    ----------
    df : Pandas DataFrame

    Notes :
    -----
    Different datasets have different naming conventions (for countries that go by multiple names and abbreviations).
    This function imposes a convention on a selection of these country names.  
    """
    # these lists are one-to-one. countries compared via manual inspection, unfortunately. 
    mismatch_labels_bad = ['Lao People\'s Democratic Republic', 'Mainland China',
                           'Occupied Palestinian Territory','Republic of Korea', 'Korea, South', 
                           'Gambia, The ', 'UK', 
                           'USA', 'Iran (Islamic Republic of)',
                           'Bahamas, The', 'Russian Federation', 'Czech Republic', 'Republic Of Ireland',
                          'Hong Kong Sar', 'Macao Sar', 'Uk','Us',
                           'Congo ( Kinshasa)','Congo ( Brazzaville)',
                           'Cote D\' Ivoire', 'Viet Nam','Guinea- Bissau','Guinea','Usa']

    mismatch_labels_good = ['Laos','China',
                            'Palestine', 'South Korea', 'South Korea', 
                            'The Gambia', 'United Kingdom', 
                            'United States','Iran',
                            'The Bahamas','Russia','Czechia','Ireland',
                            'Hong Kong','Macao','United Kingdom', 'United States',
                            'Democratic Republic Of The Congo','Republic Of The Congo',
                            'Ivory Coast','Vietnam', 'Guinea Bissau','Guinea Bissau','United States']
    
    # three cases, column names, country names, or datetime. 
    if category == 'location':
        reformatted_values = []
        for val in values_to_transform:
            reformatted_values.append(' '.join(re.sub('([A-Z][a-z]+)', r' \1', 
                                                        re.sub('([A-Z]+)|_|\/|\)|\(', r' \1', val).lower())
                                                        .split()).title())
        transformed_values = pd.Series(reformatted_values).replace(to_replace=mismatch_labels_bad, value=mismatch_labels_good)
    
    elif category == 'columns':
        reformatted_values = []
        for val in values_to_transform:
            reformatted_values.append('_'.join(re.sub('([A-Z][a-z]+)', r' \1', 
                                                     re.sub('([A-Z]+)|_|\/|\)|\(', r' \1', val)
                                                            .lower()).split()))
        transformed_values = pd.Series(reformatted_values)
        
    elif category == 'date':
        transformed_values = pd.to_datetime(pd.Series(
            values_to_transform), errors='coerce',format=dateformat).dt.normalize()


    return transformed_values

def add_time_indices(data_table):
    n_cases_pos = data_table.loc[:,'new_cases'].replace(to_replace=0, value=np.nan).dropna().reset_index()
    country_groupby_indices = country_groupby_indices_list(data_table)
    country_groupby_indices_dropped_nan = country_groupby_indices_list(n_cases_pos)
    days_since = []
    for i, c in enumerate(country_groupby_indices_dropped_nan):
        nonzero_list = list(range(len(c)))
        zero_list = 0*np.array(list(range(len(country_groupby_indices[i])-len(c))))
        days_since += list(zero_list)+nonzero_list

    data_table.loc[:, 'time_index'] = days_since
    data_table.loc[:, 'date_proxy'] = len(data_table.location.unique())*list(range(len(data_table.date.unique())))
    return data_table

## Data <a id='imports'></a>

Differences in reporting
Differences in government responses
differences in time series.

testing vs cases vs deaths
log-log plot for current growth trends
bollinger bands
histogram of trending upwards, flat, downwards
tools, different plots, correlation plots scatter matrix plots. 

Going to remove microstates like San Marino. 

data = pd.read_csv('eda_data.csv', index_col=0)
data.sample(5)

data.set_index(['location', 'date']).isna().groupby(level=1).sum().loc[:, column_name_search(data, 'new_test')].iloc[:,[0,-1]].plot()

Estimation of actual death rates, assuming 50% of people are asymptomatic and do not actually get tested.

today_data = data[data.date_proxy == data.date_proxy.max()]

today_data.loc[:, 'estimation_death_rate'] = 100 * today_data.loc[:, 'deaths_jhucsse'].values / (2. * today_data.loc[:, 'cases_jhucsse'].values)

(weighted_dr).plot.hist(bins=50)

data.loc[:,column_name_search(data, 'long').tolist() + column_name_search(data, 'lat').tolist()].iloc[:, [1,3]].plot.hist()

data.loc[:, 'population_owid'] = data.loc[:, 'population_owid'].fillna(method='bfill').fillna(method='ffill')
data.loc[:, 'population_density_owid'] = data.loc[:, 'population_density_owid'].fillna(method='bfill').fillna(method='ffill')

Reporting inconsistencies

data.loc[:, 'new_cases_per_million_owid'] = data.loc[:, 'new_cases_owid'] / (data.loc[:, 'population_owid'] / 1000000.)
data.loc[:, 'new_cases_per_million_ttc'] = data.loc[:, 'new_cases_ttc'] / (data.loc[:, 'population_owid'] / 1000000.)

ax = (data.groupby('location').sum().loc[:, ['new_cases_per_million_ttc','new_cases_per_million_owid']].diff(axis=1)).plot(label='tt')
# data.groupby('location').sum().loc[:, 'new_cases_per_million_owid'].plot(ax=ax, label='owid')
plt.legend()

# # key words account for multilevel indices
# data = pd.read_csv('data.csv', index_col=[0,1], header=[0,1])
# data.sample(5)

## Feature production <a id='newfeatures'></a>

function ```append_rolling_values``` is not working. Need to compute rolling averages for each
countries time series' individually but want to store them in the multi index DataFrame. 



Before interpolation and backfilling, I used to prune countries which did not have cases prior
to responses (i.e. "early responders" were not included)
To make my life easier, I'm only taking data which had cases before all government mandates so the rates before and after are well defined. We can think of these as being "late responders"

initial_response_data = data.replace(to_replace=[0,0.], value=np.nan)

time_dependent= model_data.set_index('location').loc[:, :'time_index'].iloc[:, :-2].reset_index()
time_independent = model_data.loc[:, 'time_index':'cvd_death_rate'].iloc[:, 1:].fillna(0)#.columns
flag_and_misc = model_data.loc[:, 'c1_flag_0':'date']

roll_widths = [3, 7, 14]
# roll_widths = list(range(2,15))


rolling_predictors = rolling_features(time_dependent, time_dependent.columns.drop('location'), roll_widths)
# This drops the first category, Afghanistan.
location_one_hot = pd.get_dummies(model_data.loc[:, 'location'], drop_first=True)
# this specifically drops the NaN category
tests_units_one_hot = pd.get_dummies(model_data.loc[:, 'tests_units'])
tests_units_one_hot.columns = 'test_units_' + reformat_values(tests_units_one_hot, category='columns')

df = pd.concat((data.loc[:, 'date_proxy':'time_index'], time_dependent.drop(columns='location'), 
                pd.concat(rolling_predictors, axis=1).reset_index(drop=True), time_independent,
                   location_one_hot, tests_units_one_hot, flag_and_misc),axis=1)

df[df.location=='United States'].loc[:, df.columns[df.columns.str.contains('new_cases')]].plot(legend=False, figsize=(10,10))

Data has the following partitions:

columns until and not including 'population' are time series variables.
        'population':'Albania' : time independent, continuous variables
        'Albania':'location' : one-hot encoded variables
        'location:' location and date, not encoded.
        
Time series variables with drift as baseline model : any columns with 'new' (cases, deaths, tests)
Time series variables with naive as baseline model : The complement to the drift baseline variables. 


## Exploratory Data Analysis<a id='EDA'></a>
Ideas for the inclusion or creation of new columns.

Moving averages
fourier
signal
flags for lots of different things

hardest hit countries

days since

extrapolated, actual, interpolated

which dataset it came from

humans view, interpret and forecast things in a way which are not available to robots. 
data driven, time dependent manner of modeling. Really trying to encapsulate the time dependence. 

government_responses = pd.concat((data.loc[:, ['location','date']],
                                  data.loc[:, column_search(data, 'oxcgrt')].drop(
                                      columns=column_search(data, 'flag')).iloc[:,2:10]), axis=1)

government_responses = regularize_names(pd.read_csv('OxCGRT_latest.csv'), locationkey='country_name')
government_responses.loc[:, 'date'] = pd.to_datetime(government_responses.loc[:, 'date'], format='%Y%m%d')
# oxcgrt_df = oxcgrt_df.set_index(['location', 'date'])
# oxcgrt_df = oxcgrt_df.drop(columns='m1_wildcard')

government_responses = pd.concat((government_responses.loc[:, ['location', 'date']], government_responses.iloc[:,3:18:2]), axis=1)

government_responses.loc[:, 'date'] = government_responses.date.dt.normalize().values



countries_with_all_responses = None
for i, country_indices in enumerate(country_groupby_indices_list(government_responses)):
    government_responses.loc[country_indices,:] = government_responses.loc[country_indices,:].fillna(method='ffill')
    has_all_flags = (government_responses.loc[country_indices,:].max() == 0).sum()
    if has_all_flags != 0:
        pass
    else:
        if countries_with_all_responses is None:
            countries_with_all_responses = government_responses.loc[country_indices,:]
        else:
            countries_with_all_responses = pd.concat((countries_with_all_responses, government_responses.loc[country_indices,:]),axis=0)

# government_responses = government_responses.dropna(axis=0).iloc[:,:10]

countries_with_all_responses.loc[:, 'date'] = pd.to_datetime(countries_with_all_responses.date).dt.normalize()

see_if_country_samples =countries_with_all_responses.groupby('location').count().sort_values(by='date') 
drop_these_countries = np.unique(np.where(~(see_if_country_samples == 148))[0])
countries_with_all_responses = countries_with_all_responses[countries_with_all_responses.location != 'Kosovo']

def add_time_indices(data_table, index_column='cases'):
    indexer = data_table.loc[:, ['location',index_column]].replace(to_replace=0, value=np.nan).dropna().reset_index()
    country_groupby_indices = country_groupby_indices_list(data_table)
    country_groupby_indices_dropped_nan = country_groupby_indices_list(indexer)
    days_since = []
    for i, c in enumerate(country_groupby_indices_dropped_nan):
        nonzero_list = list(range(len(c)))
        zero_list = 0*np.array(list(range(len(country_groupby_indices[i])-len(c))))
        days_since += list(zero_list)+nonzero_list

    data_table.loc[:, 'time_index'] = days_since
    data_table.loc[:, 'date_proxy'] = len(data_table.location.unique())*list(range(len(data_table.date.unique())))
    return data_table

response_ranges = []
for country_indices in country_groupby_indices_list(countries_with_all_responses):
    tmp = countries_with_all_responses.loc[country_indices,:].replace(to_replace=[0,0.], value=np.nan)
    for c in tmp.columns[2:]:
        active_range = tmp.set_index('date').loc[:,c].dropna()
        response_ranges.append(pd.IndexSlice[active_range.index.min():active_range.index.max()])

response_slices_df = pd.DataFrame(np.array(response_ranges).reshape(countries_with_all_responses.location.nunique(), -1))

country_list = []
slice_list = []

for j, (country, country_df) in enumerate(all_responses.groupby(level=0)):
    active_dates = country_df.replace(to_replace=0., value=np.nan)
    country_list += [country]
    before_list = []
    after_list = []
    for i, single_response in enumerate(active_dates.columns):
        effective_range = active_dates[single_response].dropna(axis=0)
        before = effective_range.reset_index().Date.min()
#         after = effective_range.reset_index().Date.max()
        slice_list += [before]   
        
enacted_ended_df = pd.DataFrame(np.array(slice_list).reshape(len(country_list), -1), index=country_list, columns=all_responses.columns)

all_responses = response_df.iloc[:, [0, 1, 2, 3, 5, 6]]
country_list = []
minmax_list = []
for j, (country, country_df) in enumerate(all_responses.groupby(level=0)):
    active_dates = country_df.replace(to_replace=0., value=np.nan)
    country_list += [country]
    for i, single_response in enumerate(active_dates.columns):
        effective_range = active_dates[single_response].dropna(axis=0)
        before = effective_range.reset_index().Date.min()
        after = effective_range.reset_index().Date.max()
        minmax_list += [before, after]   

start_end_columns = np.array([[x+'_start', x+'_end'] for x in all_responses.columns.tolist()]).ravel()
start_end_df = pd.DataFrame(np.array(minmax_list).reshape(len(country_list), -1), index=country_list, columns=start_end_columns)
start_end_filtered_df = start_end_df.drop(columns=['Close_public_transport_start','Close_public_transport_end']).dropna(axis=0)
filtered_countries = start_end_filtered_df.index
enacted_ended_filtered_df = enacted_ended_df.drop(columns=['Close_public_transport']).loc[filtered_countries, :]
start_end_filtered_df = start_end_df.drop(columns=['Close_public_transport_start','Close_public_transport_end']).dropna(axis=0)
filtered_countries = start_end_filtered_df.index
enacted_ended_filtered_df = enacted_ended_df.drop(columns=['Close_public_transport']).loc[filtered_countries, :]

first_response_dates = start_end_df.min(axis=1).sort_index()
first_response_dates.head(10)

first_case_dates = test_multiindex_df.reset_index(level=1).groupby(level=0).Date.min().sort_index()
first_case_dates.head(10)

dates_with_test_data = test_multiindex_df.tests_cumulative.dropna()
dates_with_test_data.head()

min_testing_dates = test_multiindex_df.tests_cumulative.dropna().reset_index(level=1).groupby(level=0).Date.min()

first_testing_dates = test_multiindex_df.tests_cumulative.dropna().reset_index(level=1).groupby(level=0).Date.min()
last_testing_dates = test_multiindex_df.tests_cumulative.dropna().reset_index(level=1).groupby(level=0).Date.max()

first_testing_dates.reset_index()

# convert entire dataframe to index so it can be used to slice testing data, dataframe
first_tmp =  first_testing_dates.reset_index().set_index(['Country','Date'])
last_tmp =  last_testing_dates.reset_index().set_index(['Country','Date'])
first_tmp.head()

test_min = test_multiindex_df.loc[first_tmp.index, :]
test_max = test_multiindex_df.loc[last_tmp.index, :]

# reset index so we can subtract datetime variables.
test_max_reset = test_max.reset_index(level=1)
test_min_reset = test_min.reset_index(level=1)
time_differential = (test_max_reset.Date - test_min_reset.Date).dt.days
testing_rates = np.log(test_max_reset.tests_cumulative / test_min_reset.tests_cumulative)# / time_intervals


test_final_test_initial_time_intervals = (test_max_reset.Date - test_min_reset.Date).dt.days

case_response_differential = (first_case_dates-first_response_dates).dt.days

late_response = case_response_differential < 0
late_response

states_to_inspect = ['Michigan', 'Georgia', 'New York', 'Texas']

dead=us_deaths[us_deaths['Province_State'].isin(states_to_inspect)].groupby(by='Province_State').sum()
confirmed=us_cases[us_cases['Province_State'].isin(states_to_inspect)].groupby(by='Province_State').sum()
confirmed.head()


since_first_case_normalized_u = u.replace(to_replace=[0,0.], value=np.nan)
since_first_case_normalized_u.loc[(states_to_inspect,'Confirmed'), :].iloc[:,6:].values
since_first_case_normalized_u.loc[(states_to_inspect,'Confirmed'), :].iloc[:,6:] / since_first_case_normalized_u.loc[(states_to_inspect,'Dead'), :].iloc[:,6:]
since_first_case_normalized_u.loc[(states_to_inspect,'Confirmed'), :].iloc[:,6:].apply(np.log10).transpose().plot()
time_series_df = since_first_case_normalized_u#.iloc[:, 6:]
death_rate_df = since_first_case_normalized_u.loc[(states_to_inspect,'Dead'), :].iloc[:,6:].copy()
death_rate_normalized = 100 * since_first_case_normalized_u.loc[(states_to_inspect,'Dead'), :].iloc[:,6:].values / since_first_case_normalized_u.loc[(states_to_inspect,'Confirmed'), :].iloc[:,6:].values
death_rate_df.loc[:, :] = death_rate_normalized


since_first_case_normalized_u.loc[(states_to_inspect,'Confirmed'), :].iloc[:,6:].values

since_first_case_normalized_u.loc[(states_to_inspect,'Confirmed'), :].iloc[:,6:] / since_first_case_normalized_u.loc[(states_to_inspect,'Dead'), :].iloc[:,6:]

since_first_case_normalized_u.loc[(states_to_inspect,'Confirmed'), :].iloc[:,6:].apply(np.log10).transpose().plot()

time_series_df = since_first_case_normalized_u#.iloc[:, 6:]


first_case_dates.astype('category').cat.codes.plot.hist(bins=50)

pd.concat(new_feature_df_list,ignore_index=False).sort_index(axis=1)

fig = plt.figure(figsize=(10,10), dpi=200)
death_rate_df.transpose().plot().legend(bbox_to_anchor=(1, 1))
_ = plt.xlabel('Date')
_ = plt.ylabel('Death Rate (%)')
plt.grid(True, axis='both')
plt.title('Death rate by state')
plt.savefig('death_rate_NY_MI_GA.png', bbox_inches='tight')



fig, (ax,ax2) = plt.subplots(1, 2, sharey=True,  figsize=(20,5), dpi=200)
confirmed.loc[:, '2/21/20':].transpose().plot(ax=ax).legend(bbox_to_anchor=(0.2, 1))
dead.loc[:, '2/21/20':].transpose().plot(ax=ax2).legend(bbox_to_anchor=(0.2, 1))
ax.set_yscale('log')
ax2.set_yscale('log')
ax.set_title('Number of confirmed cases vs. time')
ax2.set_title('Number of diseased vs. time')
ax.grid(True, axis='both')
ax2.grid(True, axis='both')
plt.savefig('cases_vs_dead_comparison_GA_NY_MI.png', bbox_inches='tight')

def top_5_counties(state_df, state_name):
    state = state_df[(state_df.Province_State==state_name)]
    state = state.drop(columns=['UID','iso2','iso3','code3','FIPS','Country_Region','Lat','Long_','Combined_Key','Province_State'])
    top5_counties = state.groupby(by='Admin2').sum().sum(axis=1).sort_values(ascending=False)[:5].index.tolist()
    state_info = state[state.Admin2.isin(top5_counties)].set_index('Admin2').transpose()
    state_info.columns.name = 'County'
    return state_info

### Global COVID data

global_recovered_dates_only = global_recovered.set_index('Country/Region').loc[:, '1/22/20':].groupby(level=0).sum()
global_confirmed_dates_only = global_confirmed.set_index('Country/Region').loc[:, '1/22/20':].groupby(level=0).sum()
global_dead_dates_only = global_dead.set_index('Country/Region').loc[:, '1/22/20':].groupby(level=0).sum()

global_dead['type']='Dead'
global_confirmed['type']='Confirmed'
global_recovered['type']='Recovered'

dead=global_dead[global_dead['Country/Region'].isin(['Germany', 'Italy', 'US'])].set_index('Country/Region').loc[:,'1/22/20':]#.iloc[:, 4:].transpose().columns
confirmed=global_confirmed[global_confirmed['Country/Region'].isin(['Germany', 'Italy', 'US'])].set_index('Country/Region').loc[:,'1/22/20':]#.iloc[:, 4:].transpose().columns

global_dead = global_dead.sort_index(axis=1)
global_confirmed = global_confirmed.sort_index(axis=1)
global_recovered = global_recovered.sort_index(axis=1)

skr = global_confirmed.groupby('Country/Region').sum().iloc[143, :].loc['1/22/20':'4/28/20']
skr.head()

top10 = global_confirmed.groupby('Country/Region').sum().loc[:, '1/22/20':'4/28/20'].sort_values(by='4/28/20').iloc[-10:, :]
skr = global_confirmed.groupby('Country/Region').sum().loc['Korea, South', '1/22/20':'4/28/20']

top10_and_south_korea = pd.concat((top10, skr.to_frame(name='South Korea').transpose()),axis=0).sort_index()

fig, ax = plt.subplots(figsize=(10,10))
for i, country_time_series in enumerate(top10_and_south_korea.replace(to_replace=[0,0.], value=np.nan).values):
    nan_count = np.sum(np.isnan(country_time_series))
    days_since_first = np.roll(country_time_series, -nan_count)
    plt.plot(days_since_first, label=top10_and_south_korea.index[i])
    
plt.legend()
plt.yscale('log')
plt.show()

global_dead_dates_only

dsum = global_dead_dates_only.sum()
csum = global_confirmed_dates_only.sum()
drsum = 100*dsum/csum
drsum.plot()
_ = plt.xlabel('Date')
_ = plt.ylabel('Death Rate (%)')
_ = plt.title('Average global death rate vs. time')
plt.grid(True, axis='both')
plt.savefig('death_rate_global.png', bbox_inches='tight')

first_case_dates = case_df.reset_index().set_index(['Country','date']).total_cases.replace(
                           to_replace=0,value=np.nan).dropna().reset_index(level=1).groupby(level=0).date.min()

first_response_dates = response_df.min(axis=1)
tmp = response_df.copy()
dt = pd.DataFrame(np.tile(first_case_dates.values.reshape(-1,1),(1, response_df.shape[1])))
diff_df = tmp - np.tile(first_case_dates.values.reshape(-1,1),(1, response_df.shape[1]))
num_miss=diff_df.where(diff_df > pd.Timedelta(days=0)).isna().sum(1).sort_values(ascending=False)
countries_with_cases_before_responses = num_miss.where(num_miss==0).dropna().index

Just using the endpoints of each interval is not going to work as well, because if the endpoints represent outliers then they
will not capture the overall trend. Therefore, I will do the following: average the two intervals before and after the quarantine measure (average the cases/((1M people)(100k tests)) and then compare the averages with the value at the quarantine date. I believe this is fair because it's being applied equally to both intervals.


data = case_multiindex_df.join(test_multiindex_df, lsuffix='_x', rsuffix='_y').sort_index(axis=1, ascending=False)

To ensure that total cases is a cumulative variable, replace zeros with np.nan and then backwards interpolate
Growth rate calculations require values greater than zero, so remove all dates where there are zero confirmed cases, per country.

# Normalize the time series, fill in with missing values with nan. 
data = data.reindex(pd.MultiIndex.from_product([data.index.levels[0], 
                    data.index.get_level_values(1).unique().sort_values()], names=['Country', 'Date']), fill_value=np.nan)

# Don't use zeros this messes things up.
data.loc[:, 'total_cases'] = data.loc[:, 'total_cases'].replace(to_replace=[0,0.], value=np.nan)
# instantiate with copy so that we can iterate over DataFrame groupby
data.loc[:, 'total_cases_interpolated'] = data.loc[:, 'total_cases'].copy()
data.loc[:, 'tests_cumulative_interpolated'] = data.loc[:, 'tests_cumulative'].copy()

for country, country_df in data.groupby(level=0):
    data.loc[country, 'total_cases_interpolated'] = country_df.loc[:, 'total_cases'].interpolate(limit_direction='backward').values
    data.loc[country, 'tests_cumulative_interpolated'] = country_df.loc[:, 'tests_cumulative'].interpolate(limit_direction='backward').values
    data.loc[country, 'population'] = country_df.loc[:, 'population'].fillna(method='backfill')

data.loc[:, 'cases_per_1M_people_per_100k_tests'] = (data.total_cases_interpolated / ((data.population/1000000.) * (data.tests_cumulative_interpolated))).values
data.loc[:, 'cases_per_1M_people'] = (data.total_cases_interpolated / ((data.population/1000000.))).values


data.loc[:, 'cumulative_normalized_case_test_ratio'] = (data.total_cases_interpolated / ((data.population/1000000.) * (data.tests_cumulative_interpolated))).cumsum().apply(np.log)

before_minus_after = response_multiindex_df.applymap(multiindex_response_date_to_average_rates).replace(to_replace=0., value=np.nan).sort_index()

before_minus_after_residual_values =  before_minus_after.values - np.tile(before_minus_after.mean(1).values.reshape(-1,1), (1, 5))
before_minus_after_residual_df = pd.DataFrame(before_minus_after_residual_values.reshape(-1, 5), columns=before_minus_after.columns, index=before_minus_after.index)
before_minus_after_residual_df.head()

data.loc[:, 'cumulative_normalized_case_test_ratio'] = (data.total_cases_interpolated / data.tests_cumulative_interpolated).cumsum()













In [ ]:
# for max_date_in_window in range(start_date, time_index.max() - n_days_into_future + 1):
#     # Take all model_data with date proxy less than numerical value, leading_window_date_not_included
#     frame_data = model_data[(time_index <= max_date_in_window) & 
#                             (time_index > max_date_in_window-frame_size)]
#     #     print(frame_data.shape)
#     # Reshape the array such that each element along axis=0 is a time series of all feature model_data of a specific country.
#     reshaped_frame_data = frame_data.values.reshape(n_countries, frame_size, -1)
#     #     print(reshaped_frame_data.shape)
#     # Truncate / pad the windows along the "time" axis, axis=1. (pad_sequences takes in an iterable of iterables;
#     # the first axis is always the default iteration axis. 
#     # *********************** WARNING: pad_sequences converts to integers by default *********************
#     resized_frame_data = pad_sequences(reshaped_frame_data, maxlen=frame_size, dtype=np.float64)
#     frame_data_4D = resized_frame_data[np.newaxis, :, :, :]
#     if max_date_in_window == start_date:
#         X = frame_data_4D.copy()
#     else:
#         X = np.concatenate((X, frame_data_4D),axis=0)

        
# y = target_data.values[-X.shape[0]:].reshape(-1,1)
# y_time_index = time_index.values[-X.shape[0]:].reshape(-1,1)
# # y = target_data.values.transpose()[-X.shape[0]:,:]
# # y_time_index = time_index.values.reshape(-1, time_index.nunique()).transpose()[-X.shape[0]:]

# y_train =  y[:-(n_validation_frames+n_test_frames),:]
# y_validate = y[-(n_validation_frames+n_test_frames):-n_test_frames, :]
# X_train= X[:-(n_validation_frames+n_test_frames),:,:,:]
# y_train =  y[:-(n_validation_frames+n_test_frames),:]
# X_validate = X[-(n_validation_frames+n_test_frames):-n_test_frames, :, :, :]
# y_validate = y[-(n_validation_frames+n_test_frames):-n_test_frames, :]
# X_test = X[-n_test_frames:, :, :, :] 
# y_test = y[-n_test_frames:, :]
# splits =  (X_train, y_train, X_validate, y_validate,
#            X_test, y_test)

# y_train_time = y_time_index[:-(n_validation_frames+n_test_frames),:]
# y_validate_time = y_time_index[-(n_validation_frames+n_test_frames):-n_test_frames, :]
# y_test_time = y_time_index[-n_test_frames:, :]

# Before normalization, plot the splits 

#     # Note that the last frame (date_range) that exists in X has already been determined by the choice of the number
#     # of steps to predict in the future, this is only slicing the frames. 
#     if train_test_only:
#         (X_train, y_train, X_test, y_test) = splits
#     else:
#         (X_train, y_train, X_validate, y_validate, X_test, y_test) = splits

# #     if normalization_method=='minmax':
#         # To encapsulate the time-dependent nature of the problem and ignore the dramatic difference between current
#         # and initial behavior, only rescale the validation and testing frames by the most recent frame's values.
#         # There is only a single value per feature in this case, meaning that to rescale, the values need to
#         # be repeated for each validation, test frame for each country for each timestep.
#     X_min = X_train.min(axis=(2))
#     X_max = X_train.max(axis=(2))


#     X_train_scaled = minmax(X_train, X_min[:, :, np.newaxis, :],
#                     X_max[:, :, np.newaxis, :])
#     X_test_scaled = minmax(X_test, X_min[-1][np.newaxis, :, np.newaxis, :], 
#                            X_max[-1][np.newaxis, :, np.newaxis, :])
#     if train_test_only:
#     # Normalize the training data by each frame's specific mean and std deviation. 
#         splits = (X_train_scaled, y_train, X_test_scaled, y_test)
#     else:
#         X_validate_scaled = minmax(X_validate, X_min[-1,:][np.newaxis, :, np.newaxis, :], 
#                                    X_max[-1,:][np.newaxis, :, np.newaxis, :])
#         splits = (X_train_scaled, y_train, X_validate_scaled, y_validate, X_test_scaled, y_test)


In [ ]:
# for max_date_in_window in range(start_date, time_index.max() - n_days_into_future + 2):
#     # Take all model_data with date proxy less than numerical value, leading_window_date_not_included
#     frame_data = model_data[(time_index <= max_date_in_window - 1) & 
#                             (time_index >= max_date_in_window-frame_size)]
#     #     print(frame_data.shape)
#     # Reshape the array such that each element along axis=0 is a time series of all feature model_data of a specific country.
#     reshaped_frame_data = frame_data.values.reshape(n_countries, frame_size, -1)
#     #     print(reshaped_frame_data.shape)
#     # Truncate / pad the windows along the "time" axis, axis=1. (pad_sequences takes in an iterable of iterables;
#     # the first axis is always the default iteration axis. 
#     # *********************** WARNING: pad_sequences converts to integers by default *********************
#     resized_frame_data = pad_sequences(reshaped_frame_data, maxlen=frame_size, dtype=np.float64)
#     frame_data_4D = resized_frame_data[np.newaxis, :, :, :]
# #     break
#     if max_date_in_window == start_date:
#         print('Starting with frame ranging time_index values:', max_date_in_window-frame_size, max_date_in_window - 1)

#         X = frame_data_4D.copy()
#     else:
#         X = np.concatenate((X, frame_data_4D),axis=0)
# print('Ending with frame ranging time_index values:', max_date_in_window-frame_size, max_date_in_window - 1)
# y = target_data.values.reshape(-1, time_index.nunique()).transpose()[-X.shape[0]:,:]


In [ ]:
# def minmax(X, X_min, X_max):
#     # X_min and X_max need to have already been made into 4-d tensors with np.newaxis
#     tile_shape = np.array(np.array(X.shape) / np.array(X_min.shape), dtype=int)
#     denominator = np.tile(X_max, tile_shape) - np.tile(X_min, tile_shape)
#     denominator[denominator==0] = 1
#     X_scaled = (X - np.tile(X_min, tile_shape)) / denominator
#     return X_scaled

# def normal(X, X_mean, X_std):
#     tile_shape = np.array(np.array(X.shape) / np.array(X_mean.shape), dtype=int)
#     mean_ = np.tile(X_mean, tile_shape)
#     std_ =  np.tile(X_std, tile_shape)   
#     std_[np.where(std_==0.)] = 1
#     X_scaled = ((X - mean_) /  std_)
#     return X_scaled

# def normalize_Xy_splits(splits, normalization_method='minmax', train_test_only=False, feature_indices=None):
#     """ Split into training, validation and test data.
#     """
#     min_, max_ = (0, 1)
#     # Note that the last frame (date_range) that exists in X has already been determined by the choice of the number
#     # of steps to predict in the future, this is only slicing the frames. 
#     if train_test_only:
#         (X_train, y_train, X_test, y_test) = splits
#     else:
#         (X_train, y_train, X_validate, y_validate, X_test, y_test) = splits

#     if normalization_method=='minmax':
#         # To encapsulate the time-dependent nature of the problem and ignore the dramatic difference between current
#         # and initial behavior, only rescale the validation and testing frames by the most recent frame's values.
#         # There is only a single value per feature in this case, meaning that to rescale, the values need to
#         # be repeated for each validation, test frame for each country for each timestep.
#         X_min = X_train.min(axis=(2))
#         X_max = X_train.max(axis=(2))


#         X_train_scaled = minmax(X_train, X_min[:, :, np.newaxis, :],
#                         X_max[:, :, np.newaxis, :])
#         X_test_scaled = minmax(X_test, X_min[-1][np.newaxis, :, np.newaxis, :], 
#                                X_max[-1][np.newaxis, :, np.newaxis, :])
#         if train_test_only:
#         # Normalize the training data by each frame's specific mean and std deviation. 
#             splits = (X_train_scaled, y_train, X_test_scaled, y_test)
#         else:
#             X_validate_scaled = minmax(X_validate, X_min[-1,:][np.newaxis, :, np.newaxis, :], 
#                                        X_max[-1,:][np.newaxis, :, np.newaxis, :])
#             splits = (X_train_scaled, y_train, X_validate_scaled, y_validate, X_test_scaled, y_test)
#     else:
        
#         X_mean = X_train.mean(axis=(1,2))
#         X_std = X_train.std(axis=(1,2))


#         X_train_scaled = normal(X_train, 
#                                 X_mean[:, np.newaxis, np.newaxis, :],
#                                 X_std[:, np.newaxis, np.newaxis, :])
#         X_test_scaled = normal(X_test, 
#                                X_mean[-1,:][np.newaxis, np.newaxis, np.newaxis, :], 
#                                X_std[-1,:][np.newaxis, np.newaxis, np.newaxis, :])
        
#         if train_test_only:
#         # Normalize the training data by each frame's specific mean and std deviation. 
#             splits = (X_train_scaled, y_train, X_test_scaled, y_test)
#         else:
#             X_validate_scaled = normal(X_test, 
#                                X_mean[-1,:][np.newaxis, np.newaxis, np.newaxis, :], 
#                                X_std[-1,:][np.newaxis, np.newaxis, np.newaxis, :])
#             splits = (X_train_scaled, y_train, X_validate_scaled, y_validate, X_test_scaled, y_test)

#     return splits

In [ ]:


# def concatenate_4d_into_3d(splits, train_test_only=False):
    
#     if train_test_only:
#         (X_train, y_train, X_test, y_test) = splits
#         X_train = np.concatenate(X_train, axis=0)
#         y_train = np.concatenate(y_train, axis=0)
#         X_test = np.concatenate(X_test, axis=0)
#         y_test = np.concatenate(y_test, axis=0)
#         concat_splits = (X_train, y_train, X_test, y_test) 
#     else:
#         (X_train, y_train, X_validate, y_validate, X_test, y_test) = splits
#         X_train = np.concatenate(X_train, axis=0)
#         y_train = np.concatenate(y_train, axis=0)
#         X_validate = np.concatenate(X_validate, axis=0)
#         y_validate = np.concatenate(y_validate, axis=0)
#         X_test = np.concatenate(X_test, axis=0)
#         y_test = np.concatenate(y_test, axis=0)
#         concat_splits = (X_train, y_train, X_validate, y_validate, X_test, y_test) 
#     return concat_splits

# def transpose_for_separable2d(splits, train_test_only=False):
#     if train_test_only:
#         (X_train, y_train, X_test, y_test) = splits
#         X_train = np.transpose(X_train, axes=[0,2,1,3])
#         X_test = np.transpose(X_test, axes=[0,2,1,3])
#         transpose_split = (X_train, y_train, X_test, y_test) 
#     else:
#         (X_train, y_train, X_validate, y_validate, X_test, y_test) = splits
#         X_train = np.transpose(X_train, axes=[0,2,1,3])
#         X_validate = np.transpose(X_validate, axes=[0,2,1,3])
#         X_test = np.transpose(X_test, axes=[0,2,1,3])
#         transpose_split = (X_train, y_train, X_validate, y_validate, X_test, y_test) 
#     return transpose_split


In [ ]:
# def n_step_model_predictions(model_data, target_data, time_index, model_generator, frame_size, start_date, n_countries,
#                              n_validation_frames, n_test_frames, predict_steps, f, k, epochs, learning_rate, batch_size,
#                              train_test_only=False, Xy_truncation=None,verbose=0, train_or_test='train'):
    
#     """ wrapper for iteration loop 
    
#     data : DataFrame of very specific make
    
#     model : one of my custom models, sequential_Conv1D_model, SeparableConv2D_model, parallel_Conv1D_model
    
    
    
#     """

#     for n_days_into_future in predict_steps:
#         X, y = create_Xy(model_data, target_data, time_index, start_date, frame_size, n_days_into_future, n_countries)
#         splits = split_Xy(X, y, frame_size, n_validation_frames, n_test_frames, date_normalization=True,
#                                   train_test_only=train_test_only)
        
#         A_splits = concatenate_4d_into_3d(splits, train_test_only=train_test_only) 
#         if train_test_only:
#             X_train, y_train, X_test, y_test = A_splits
#             X_validate, y_validate = X_test, y_test
#         else:
#             X_train, y_train, X_validate, y_validate, X_test, y_test = A_splits

#         model = model_generator(X_train, f, k)
#         model.compile(loss='mae', optimizer=Adam(learning_rate=learning_rate))
#         # fit network
#         history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_validate, y_validate), 
#                   batch_size=batch_size, verbose=verbose)
        
#         ### analysis
#         if train_or_test == 'train':
#             y_true = y_validate.ravel()
#             y_naive = y[-(n_validation_frames+n_test_frames+n_days_into_future):-(n_test_frames+n_days_into_future), :].ravel()
#             y_predict = model.predict(X_validate).ravel()
#             # evaluate model
#         else:
#             y_true = y_test.ravel()
#             y_naive = y[-(n_test_frames+n_days_into_future):-n_days_into_future, :]
#             y_predict = model.predict(X_test).ravel()

#     return y_test, y_naive, y_predict, model, history, A_splits

In [ ]:

def create_Xy(model_data, target_data, time_index, start_date, frame_size, n_days_into_future, n_countries):
    # can't include the max date because need at least 1 day in future to predict. +1 because of how range doesn't include endpoint
    for max_date_in_window in range(start_date, time_index.max() - n_days_into_future + 1):
        # Take all model_data with date proxy less than numerical value, leading_window_date_not_included
        frame_data = model_data[(time_index <= max_date_in_window) & 
                                (time_index > max_date_in_window-frame_size)]
        #     print(frame_data.shape)
        # Reshape the array such that each element along axis=0 is a time series of all feature model_data of a specific country.
        reshaped_frame_data = frame_data.values.reshape(n_countries, frame_size, -1)
        #     print(reshaped_frame_data.shape)
        # Truncate / pad the windows along the "time" axis, axis=1. (pad_sequences takes in an iterable of iterables;
        # the first axis is always the default iteration axis. 
        # *********************** WARNING: pad_sequences converts to integers by default *********************
        resized_frame_data = pad_sequences(reshaped_frame_data, maxlen=frame_size, dtype=np.float64)
        frame_data_4D = resized_frame_data[np.newaxis, :, :, :]
        if max_date_in_window == start_date:
            X = frame_data_4D.copy()
        else:
            X = np.concatenate((X, frame_data_4D),axis=0)
    y = target_data.values.reshape(-1, time_index.nunique()).transpose()[-X.shape[0]:,:]
    # y = model_data.new_cases_weighted.values.reshape(-1, model_data.time_index.nunique()).transpose()[-X.shape[0]:,:]
    return X, y

def split_Xy(X, y, frame_size, n_validation_frames, n_test_frames, date_normalization=True,
                          train_test_only=False):
    """ Split into training, validation and test data.
    """

    # Note that the last frame (date_range) that exists in X has already been determined by the choice of the number
    # of steps to predict in the future, this is only slicing the frames. 
    if train_test_only:
        X_train= X[:-n_test_frames,:,:,:]
        y_train =  y[:-n_test_frames,:]
        X_test = X[-n_test_frames:, :, :, :] 
        y_test = y[-n_test_frames:, :]
        splits =  (X_train, y_train, X_test, y_test)
    else:
        X_train= X[:-(n_validation_frames+n_test_frames),:,:,:]
        y_train =  y[:-(n_validation_frames+n_test_frames),:]
        X_validate = X[-(n_validation_frames+n_test_frames):-n_test_frames, :, :, :]
        y_validate = y[-(n_validation_frames+n_test_frames):-n_test_frames, :]
        X_test = X[-n_test_frames:, :, :, :] 
        y_test = y[-n_test_frames:, :]
        splits =  (X_train, y_train, X_validate, y_validate,
                   X_test, y_test)
                   
    return splits



In [ ]:
def n_day_forecasting(data, n_test_days, n_days_into_future, col_transformer=StandardScaler(), n_prune=2, train_or_test='train'):
    # X has already been shifted by n_days_into future, so the last day in X is predicting that last day + n_days_into_future. 
    #     y_naive = X.new_cases_weighted[X.time_index > X.time_index.max() - n_days_into_future]
    n_countries = data.location.nunique()
    chronological_data = data.sort_values(by=['time_index','location'])
    # chronological_data = chronological_data[chronological_data.days_since_first_case >0].reset_index(drop=True)

    X = chronological_data[(chronological_data.time_index 
                            <=  chronological_data.time_index.max() - n_days_into_future)].reset_index(drop=True)
    y = chronological_data[(chronological_data.time_index  
                            >= n_days_into_future)].loc[:, ['time_index','new_cases_weighted']].reset_index(drop=True)

    train_indices = X[X.time_index <= X.time_index.max() - n_test_days].index
    test_indices = X[X.time_index > X.time_index.max() - n_test_days].index

    # 2 slices date, location, 4 slices date,location,time_index, days_since_first_case
    X_train = X.iloc[train_indices, n_prune:]#.apply(lambda x : np.log(x+1))
    y_train = y.loc[train_indices]

    X_test = X.iloc[test_indices, n_prune:]#.apply(lambda x : np.log(x+1))
    y_test =  y.loc[test_indices]

    _ = col_transformer.fit(X_train)
    X_train_normalized =  col_transformer.transform(X_train)
    X_test_normalized =  col_transformer.transform(X_test)

    n_step_models = []

    model = Ridge(alpha=0.01, fit_intercept=False, tol=1e-12) 
    _ = model.fit(X_train_normalized, y_train)
    
    if train_or_test == 'train':
        y_naive = X_train.new_cases_weighted
        y_true, y_predict, mae = classifier_analysis(model, X_train_normalized, y_train, plot=False, metric='mae')
    else:
        y_naive = X_test.new_cases_weighted
        y_true, y_predict, mae = classifier_analysis(model, X_test_normalized, y_test, plot=False, metric='mae')

        
    return  (X_train_normalized, X_test_normalized), (y_true, y_naive, y_predict), model 

In [ ]:

# def normalize_Xy_splits(splits, feature_range=(0., 0.5), normalization_method='minmax',
#                         train_test_only=False, feature_indices=None):
#     """ Split into training, validation and test data.
#     """
#     min_, max_ = (0, 0.5)
#     (X_train, y_train, X_validate, y_validate, X_test, y_test) = splits
#     for i in range(1, X_train.shape[0]+1):
#         # find the minima and maxima of all features for all countries, ranging up to current frame and 
#         # each time step in the frame. 
#         up_to_current_frame_min = X_train[:i,:,:,:].min((0,1,2))
#         up_to_current_frame_max = X_train[:i,:,:,:].max((0,1,2))
#         latest_min_array = np.tile(up_to_current_frame_min[np.newaxis, np.newaxis, np.newaxis, :],(1,n_countries,frame_size,1))
#         latest_max_array = np.tile(up_to_current_frame_max[np.newaxis, np.newaxis, np.newaxis, :],(1,n_countries,frame_size,1))
#         if i == 1:
#             frame_min_array = latest_min_array
#             frame_max_array = latest_max_array
#         else:
#             frame_min_array = np.concatenate((frame_min_array, 
#                                                    latest_min_array)
#                                                   ,axis=0)
#             frame_max_array = np.concatenate((frame_max_array, 
#                                                    latest_max_array)
#                                                   ,axis=0)

#     # frame_min_array = np.tile(frame_min_array, (1, n_countries, 1, 1))
#     # frame_max_array = np.tile(frame_max_array, (1, n_countries, 1, 1))

#     minmax_denominator = (frame_max_array-frame_min_array)
#     minmax_denominator[np.where(minmax_denominator==0)]=1
#     X_train_scaled = (max_-min_)*(X_train - frame_min_array) / minmax_denominator
#     # Use the latest min and max for test scaling.

#     latest_minmax_denominator = latest_max_array - latest_min_array
#     latest_minmax_denominator[np.where(latest_minmax_denominator==0)] = 1
#     X_validate_scaled = (max_- min_)*((X_validate - np.tile(latest_min_array,(n_validation_frames,1,1,1)))
#                                         / np.tile(latest_minmax_denominator,(n_validation_frames,1,1,1)))
#     X_test_scaled = (max_- min_)*((X_test - np.tile(latest_min_array,(n_test_frames,1,1,1))) 
#                                         / np.tile(latest_minmax_denominator,(n_test_frames,1,1,1)))
#     scaled_splits = (X_train_scaled, y_train, X_validate_scaled, y_validate, X_test_scaled, y_test) 

#     train_norm_arrays =  (frame_max_array, frame_min_array, minmax_denominator)
#     validate_and_test_norm_arrays = (latest_max_array,latest_min_array,latest_minmax_denominator)

#     return scaled_splits, train_norm_arrays, validate_and_test_norm_arrays




In [ ]:
# train_tile_shape = np.array(np.array(X_cnn_train.shape)/np.array(frame_denom.shape),int)
# minmax_inverse_train = X_train - (((X_cnn_train / 0.5) * np.tile(frame_denom, train_tile_shape)) + np.tile(frame_min, train_tile_shape))


# validate_tile_shape = np.array(np.array(X_validate.shape)/np.array(latest_denom.shape),int)
# minmax_inverse_validate = X_validate - (((X_cnn_validate / 0.5) * np.tile(latest_denom,validate_tile_shape)) + np.tile(latest_min,validate_tile_shape))


# test_tile_shape = np.array(np.array(X_cnn_test.shape)/np.array(latest_denom.shape),int)
# minmax_inverse_test = X_test - (((X_cnn_test / 0.5) * np.tile(latest_denom, test_tile_shape)) + np.tile(latest_min, test_tile_shape))
# print(np.linalg.norm(minmax_inverse_train),np.linalg.norm(minmax_inverse_validate),np.linalg.norm(minmax_inverse_test))

In [ ]:
# validate_tile_shape = np.array(np.array(X_validate.shape)/np.array(latest_denom.shape),int)
# X_validate - (((X_cnn_validate / 0.5) * np.tile(latest_denom,validate_tile_shape)) + np.tile(latest_min,validate_tile_shape))
# (latest_max, latest_min, latest_denom) = latest_minmax

# print(X_regression_train.max(),X_regression_validate.max(),X_regression_test.max())

# print(X_regression_train.shape,X_regression_validate.shape,X_regression_test.shape)

# print(y_regression_train.shape,y_regression_validate.shape,y_regression_test.shape)

In [2]:
# train_tile_shape = np.array(np.array(X_regression_train.shape) / np.array(frame_denom.shape),int)
# minmax_inverse_train = X_train - (((X_regression_train / 0.5) * np.tile(frame_denom, train_tile_shape)) + np.tile(frame_min, train_tile_shape))
# np.linalg.norm(minmax_inverse_train)

# validate_tile_shape = np.array(np.array(X_validate.shape)/np.array(latest_denom.shape),int)
# minmax_inverse_validate = X_validate - (((X_regression_validate / 0.5) * np.tile(latest_denom,validate_tile_shape)) + np.tile(latest_min,validate_tile_shape))
# np.linalg.norm(minmax_inverse_validate)

# test_tile_shape = np.array(np.array(X_regression_test.shape)/np.array(latest_denom.shape),int)
# minmax_inverse_test = X_test - (((X_regression_test / 0.5) * np.tile(latest_denom, test_tile_shape)) + np.tile(latest_min, test_tile_shape))
# np.linalg.norm(minmax_inverse_test)

In [ ]:
# epochs = 10000
# learning_rate = 0.001
# kernel = 4
# N = 8
# FC = 8
# batch_size = X_cnn_train.shape[0]

# f1, f2 = 64, 8
# k1, k2 = 4, 4

# cnn_model = Sequential()
# # kernel_initializer0=RandomUniform(minval=0.0, 
# #                                  maxval= 1.0,
# #                                  seed=0,
# #                                 dtype=float)

# cnn_model.add(Conv1D(filters=int(f1), kernel_size=int(k1),
#                  padding='valid',
#                  input_shape=X_cnn_train.shape[2:],
#                  use_bias=False,
# #                 kernel_constraint=non_neg(),
# #                  activation='relu',
# #                 kernel_initializer=kernel_initializer0
#                 )
#          )

# # cnn_model.add(A/veragePooling1D(pool_size=2))
# # cnn_model.add(Activation('relu'))
# # kernel_initializer1=RandomUniform(minval=0.0, 
# #                                  maxval= 1.0,
# #                                  seed=0,
# #                                 dtype=float)

# cnn_model.add(Conv1D(filters=int(f2), 
#                  kernel_size=int(k2), 
#                  padding='valid',
#                  use_bias=False,
# #                  kernel_constraint=non_neg(),

# #                  activation='relu',
# #                 kernel_initializer=kernel_initializer1
# #                   use_bias=False
#                 )
#          )


# # cnn_model.add(AveragePooling1D(pool_size=2))
# # cnn_model.add(Activation('relu'))
# cnn_model.add(Flatten())

# # kernel_initializer2=RandomUniform(minval=0.0, 
# #                                  maxval= 1.0,
# #                                  seed=2,
# #                                 dtype=float)

# cnn_model.add(Dense(cnn_model.output.shape[1], 
# #                 activation='relu',
#                 use_bias=False,
# #                  kernel_constraint=non_neg(),
# #                 kernel_initializer=kernel_initializer2
#                )
#          )

# # kernel_initializer=RandomUniform(minval=0.0, 
# #                                  maxval= 1.0,
# #                                  seed=0,
# #                                 dtype=float)

# cnn_model.add(Dense(1, 
#                     activation='relu',
#                     use_bias=False,
# #                      kernel_constraint=non_neg(),
# #                     kernel_initializer=kernel_initializer
#                    ))
# cnn_model.compile(loss='mse', optimizer=Adam(learning_rate=learning_rate))

In [ ]:
# train_tile_shape = np.array(np.array(X_cnn_train.shape)/np.array(frame_denom.shape),int)
# minmax_inverse_train = X_train - (((X_cnn_train / 0.5) * np.tile(frame_denom, train_tile_shape)) + np.tile(frame_min, train_tile_shape))


# validate_tile_shape = np.array(np.array(X_validate.shape)/np.array(latest_denom.shape),int)
# minmax_inverse_validate = X_validate - (((X_cnn_validate / 0.5) * np.tile(latest_denom,validate_tile_shape)) + np.tile(latest_min,validate_tile_shape))


# test_tile_shape = np.array(np.array(X_cnn_test.shape)/np.array(latest_denom.shape),int)
# minmax_inverse_test = X_test - (((X_cnn_test / 0.5) * np.tile(latest_denom, test_tile_shape)) + np.tile(latest_min, test_tile_shape))
# print(np.linalg.norm(minmax_inverse_train),np.linalg.norm(minmax_inverse_validate),np.linalg.norm(minmax_inverse_test))

# 2 nonneg

1-step MSE [Naive, CNN model] = [170.80669257142873,390.12602832774525]
1-step R^2 [Naive, CNN model] = [0.2994526992709322,0.6970863460225101]

# 2 nonneg, pool, relu
There were 0 negative predictions
1-step MSE [Naive, CNN model] = [170.80669257142873,13586.662088]
1-step R^2 [Naive, CNN model] = [0.2994526992709322,0.0]

# 2 nonneg, pool

There were 0 negative predictions
1-step MSE [Naive, CNN model] = [170.80669257142873,556.4749063236223]
1-step R^2 [Naive, CNN model] = [0.2994526992709322,0.4443304826760618]


# pool + only 1 nonneg and relu at very end. probably could use more than 1500 epochs

There were 0 negative predictions
1-step MSE [Naive, CNN model] = [170.80669257142873,476.8907287109961]
1-step R^2 [Naive, CNN model] = [0.2994526992709322,0.5434063478478779]



# negative, pooling, relu at very end

There were 0 negative predictions
1-step MSE [Naive, CNN model] = [170.80669257142873,256.32871004595506]
1-step R^2 [Naive, CNN model] = [0.2994526992709322,0.6796489686915346]


# only nonneg and relu at very end.

There were 0 negative predictions
1-step MSE [Naive, CNN model] = [170.80669257142873,138.01389460401262]
1-step R^2 [Naive, CNN model] = [0.2994526992709322,0.5543860897517913]


# only relu at very end.

There were 0 negative predictions
1-step MSE [Naive, CNN model] = [170.80669257142873,140.32871993958562]
1-step R^2 [Naive, CNN model] = [0.2994526992709322,0.5113786900493877]

# 2 dense relu

There were 0 negative predictions
1-step MSE [Naive, CNN model] = [170.80669257142873,216.5053047822524]
1-step R^2 [Naive, CNN model] = [0.2994526992709322,0.4716586801209258]

# 2 nonneg, 2 relu

There were 0 negative predictions
1-step MSE [Naive, CNN model] = [170.80669257142873,396.11096463299015]
1-step R^2 [Naive, CNN model] = [0.2994526992709322,0.6519816604307848]

# average pooling, relu at very end.

There were 0 negative predictions
1-step MSE [Naive, CNN model] = [170.80669257142873,426.28375726972956]
1-step R^2 [Naive, CNN model] = [0.2994526992709322,0.6618604718747367]

# elimination round



# 2 nonneg and relu at very end.


# only nonneg and relu at very end.

There were 0 negative predictions
1-step MSE [Naive, CNN model] = [170.80669257142873,139.36171575910947]
1-step R^2 [Naive, CNN model] = [0.2994526992709322,0.5208973986926813]

There were 0 negative predictions
1-step MSE [Naive, CNN model] = [170.80669257142873,138.42094987360994]
1-step R^2 [Naive, CNN model] = [0.2994526992709322,0.5469134697530336]

There were 0 negative predictions
1-step MSE [Naive, CNN model] = [170.80669257142873,140.77506300920305]
1-step R^2 [Naive, CNN model] = [0.2994526992709322,0.5274845291582406]

# only  relu at very end.

There were 0 negative predictions
1-step MSE [Naive, CNN model] = [170.80669257142873,140.42387636891903]
1-step R^2 [Naive, CNN model] = [0.2994526992709322,0.5162934749412229]

There were 0 negative predictions
1-step MSE [Naive, CNN model] = [170.80669257142873,141.2069144587355]
1-step R^2 [Naive, CNN model] = [0.2994526992709322,0.5047534149090478]

In [ ]:
# model_data = data.iloc[:,2:].copy()
# new_cases_index = column_search(model_data,'new_cases_per_million', threshold='match', return_style='iloc')[0]
# n_countries = data.location.nunique()
# target_data = data.new_cases_per_million
# time_index = data.time_index.astype(int)
# frame_size = 28
# start_date = frame_size #+ time_index.min()
# # start_date = 50
# n_validation_frames = 7
# n_test_frames = 1
# n_days_into_future = 1
# train_or_test = 'train'

# frame_data = model_data[(time_index <= max_date_in_window-1) & 
#                         (time_index >= max_date_in_window-frame_size)]
# #     print(frame_data.shape)
# # Reshape the array such that each element along axis=0 is a time series of all feature model_data of a specific country.
# reshaped_frame_data = frame_data.values.reshape(n_countries, frame_size, -1)

# X, y = create_Xy(model_data, target_data, time_index, start_date, frame_size, n_days_into_future, n_countries)
# n_features = X.shape[-1]
# last_day_new_cases_index = np.ravel_multi_index([[frame_size-1],[new_cases_index]],(frame_size, n_features))
# splits, indices = split_Xy(X, y, frame_size, n_validation_frames, n_test_frames,model_type='ridge')
# (train_indices, validate_indices, test_indices) = indices

# (X_train, y_train, X_validate,
#  y_validate, X_test, y_test) = splits

# X_for_naive_slicing = np.concatenate(X.reshape(X.shape[0], X.shape[1], -1), axis=0)
# y_train_naive = X_for_naive_slicing[train_indices, last_day_new_cases_index]
# y_validate_naive =  X_for_naive_slicing[validate_indices, last_day_new_cases_index]
# y_test_naive =  X_for_naive_slicing[test_indices, last_day_new_cases_index]
# # y_train_naive = (np.exp(X_for_naive_slicing[train_indices, last_day_new_cases_index])-1).ravel()
# # y_validate_naive =  (np.exp(X_for_naive_slicing[validate_indices, last_day_new_cases_index])-1).ravel()
# # y_test_naive =  (np.exp(X_for_naive_slicing[test_indices, last_day_new_cases_index])-1).ravel()



# flat_splits = flatten_Xy_splits(splits)
# (X_regression_train, y_regression_train, X_regression_validate,
#  y_regression_validate, X_regression_test, y_regression_test) = flat_splits

# X_regression = np.concatenate((X_regression_train, X_regression_validate),axis=0)
# y_regression = np.concatenate((y_regression_train, y_regression_validate),axis=0)

# scorer = make_scorer(mean_squared_error, greater_is_better=False)
# regression_model = RidgeCV(scoring=scorer, cv=[[train_indices, validate_indices]]) 
# _ = regression_model.fit(X_regression, y_regression)

In [1]:
# # The normalization is alot easier if I split into train, validate, test but this makes other parts more annoying
# scaled_splits, frame_minmax, latest_minmax =  normalize_Xy_splits(splits, feature_range=(0,1), 
#                                                                   normalization_method='minmax',
#                                                                   train_test_only=False,
#                                                                   feature_indices=None)

# (latest_max, latest_min, latest_denom) = latest_minmax
# (frame_max, frame_min, frame_denom) = frame_minmax

# flat_splits = flatten_Xy_splits(scaled_splits)
# (X_regression_scaled_train, y_regression_scaled_train, X_regression_scaled_validate,
#  y_regression_scaled_validate, X_regression_scaled_test, y_regression_scaled_test) = flat_splits

# X_regression_scaled = np.concatenate((X_regression_scaled_train, X_regression_scaled_validate),axis=0)
# y_regression_scaled = np.concatenate((y_regression_scaled_train, y_regression_scaled_validate),axis=0)

# scorer = make_scorer(mean_squared_error, greater_is_better=False)
# regression_scaled_model = RidgeCV(scoring=scorer, cv=[[train_indices, validate_indices]]) 
# _ = regression_scaled_model.fit(X_regression_scaled, y_regression_scaled)

# y_predict_train = regression_scaled_model.predict(X_regression_scaled_train).ravel()
# model_analysis(y_regression_scaled_train, y_train_naive, y_predict_train, n_countries,
# title='Ridge', suptitle='Scaled predictor training set performance')

# y_predict_validate = regression_scaled_model.predict(X_regression_scaled_validate).ravel()
# model_analysis(y_regression_scaled_validate, y_validate_naive, y_predict_validate, 
# n_countries, title='Ridge', suptitle='Scaled predictor validation set performance')

# y_predict_validate = regression_scaled_model.predict(X_regression_scaled_validate).ravel()
# model_analysis(y_regression_scaled_validate, y_validate_naive, y_predict_validate,
# n_countries, title='Ridge', suptitle='Scaled predictor validation set performance')


# # The normalization is alot easier if I split into train, validate, test but this makes other parts more annoying
# scaled_splits, frame_minmax, latest_minmax =  normalize_Xy_splits(splits, feature_range=(0,1), 
#                                                                   normalization_method='minmax',
#                                                                   train_test_only=False,
#                                                                   feature_indices=None)

# (latest_max, latest_min, latest_denom) = latest_minmax
# (frame_max, frame_min, frame_denom) = frame_minmax

# flat_splits = flatten_Xy_splits(scaled_splits)
# (X_regression_scaled_train, y_regression_scaled_train, X_regression_scaled_validate,
#  y_regression_scaled_validate, X_regression_scaled_test, y_regression_scaled_test) = flat_splits

# X_regression_scaled = np.concatenate((X_regression_scaled_train, X_regression_scaled_validate),axis=0)
# y_regression_scaled = np.concatenate((y_regression_scaled_train, y_regression_scaled_validate),axis=0)

# scorer = make_scorer(mean_squared_error, greater_is_better=False)
# regression_scaled_model = RidgeCV(scoring=scorer, cv=[[train_indices, validate_indices]]) 
# _ = regression_scaled_model.fit(X_regression_scaled, y_regression_scaled)

# y_predict_train = regression_scaled_model.predict(X_regression_scaled_train).ravel()
# model_analysis(y_regression_scaled_train, y_train_naive, y_predict_train, n_countries, title='Ridge', suptitle='Scaled predictor training set performance')

# y_predict_validate = regression_scaled_model.predict(X_regression_scaled_validate).ravel()
# model_analysis(y_regression_scaled_validate, y_validate_naive, y_predict_validate, n_countries, title='Ridge', suptitle='Scaled predictor validation set performance')

# y_predict_validate = regression_scaled_model.predict(X_regression_scaled_validate).ravel()
# model_analysis(y_regression_scaled_validate, y_validate_naive, y_predict_validate, n_countries, title='Ridge', suptitle='Scaled predictor validation set performance')

# train_or_test = 'train'
# n_test_days = 1
# n_days_into_future = 14
# n_prune = 4
# # X has already been shifted by n_days_into future, so the last day in X is predicting that last day + n_days_into_future. 
# #     y_naive = X.new_cases_weighted[X.time_index > X.time_index.max() - n_days_into_future]
# n_countries = data.location.nunique()
# model_data = full_data
# # model_data = data.drop(columns=column_search(data, 'test'))

# chronological_data = model_data.sort_values(by=['time_index','location'])

# X = chronological_data[(chronological_data.time_index 
#                         <=  chronological_data.time_index.max() - n_days_into_future)].reset_index(drop=True)

# y = chronological_data[(chronological_data.time_index  
#                         >= n_days_into_future)].loc[:, ['time_index','new_cases_weighted']].reset_index(drop=True)

# # X = X[(X.days_since_first_case > 0) & (X.time_index > 40)]
# # X = X[(X.days_since_first_case > 0)]
# # X = X[X.time_index > 40]

# y = y.loc[X.index,:]
# train_indices = X[X.time_index <= X.time_index.max() - n_test_days].index
# test_indices = X[X.time_index > X.time_index.max() - n_test_days].index

# pd.DataFrame(data[data.time_index.isin(list(range(7)))].values.reshape(-1, 7*24))

# data.set_index('time_index')#.transpose()#[list(range(7))]#.transpose()

# chronological_data.set_index('time_index').transpose()[list(range(7))].transpose()

# chronological_data.set_index('time_index').transpose()

# # 2 slices date, location, 4 slices date,location,time_index, days_since_first_case

# X_train = X.loc[train_indices, :]#.apply(lambda x : np.log(x+1))
# y_train = y.loc[train_indices,['time_index','new_cases_weighted']]

# X_test = X.loc[test_indices,:]#.apply(lambda x : np.log(x+1))
# y_test =  y.loc[test_indices,['time_index','new_cases_weighted']]#.values.ravel()

# # if train_or_test == 'train':
# y_train_naive = X_train.loc[:, ['time_index','new_cases_weighted']]
# # else:
# y_test_naive = X_test.loc[:, ['time_index','new_cases_weighted']]
# X_train = X_train.iloc[:,n_prune:]
# X_test = X_test.iloc[:,n_prune:]
# col_transformer = MinMaxScaler()
# _ = col_transformer.fit(X_train)
# X_train_normalized =  col_transformer.transform(X_train)
# X_test_normalized =  col_transformer.transform(X_test)

In [ ]:
# fold_size = 7
# folds = [X.date.unique()[fold_size*i:fold_size*(i+1)] for i in range(1 + len(X.date.unique())//fold_size)]
# folds_indices = [X[X.date.isin(fold)].index for fold in folds]
# folds_series = pd.Series(folds_indices).to_frame(name='folds')

# fold_indices = folds_series.values.flatten()[:-2]
# test_indices = folds_series.values.flatten()[-2:]
# test_indices = test_indices[0].append(test_indices[1]).sort_values()

# train_indices = fold_indices[0]
# for indices in fold_indices[1:]:
#     train_indices = full_index.union(indices)

# for i, epochs in enumerate(folds_series.values.flatten()):
#     start, end = model_data.loc[epochs, 'date'].min(), model_data.loc[epochs, 'date'].max()
#     print('Epoch {} spans the dates {} to {}'.format(i,start,end))

# start, end = model_data.loc[train_indices, 'date'].min(), model_data.loc[train_indices, 'date'].max()
# print('Training data spans the dates {} to {}'.format(start,end))

# start, end = model_data.loc[test_indices, 'date'].min(), model_data.loc[test_indices, 'date'].max()
# print('Testing data spans the dates {} to {}'.format(start, end))